In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm

In [2]:
song_info_file = "song_infos.jsonl"
with open(song_info_file, "r", encoding='utf-8') as f:
    lines = f.readlines()
    
song_infos = [json.loads(_) for _ in lines]
song_infos = {_["song_id"]:_ for _ in song_infos }
song_infos["34808761"]

{'song_id': '34808761',
 'song_name': '너를 위한 단어 (It’s Yours)',
 'artist_name': 'NCT DREAM',
 '앨범': 'Glitch Mode - The 2nd Album',
 'album_num': '10901349',
 '발매일': '2022.03.28',
 '장르': 'R&B/Soul',
 'lyric': "Girl 널 닮은 단어가 참 많아\n내 하루 속에\n몇 번씩 스치듯 널 만나\n어제 읽은 책\n또 좀 전에 같이 본 Romance 영화\nMy favorite song 모두 네가 있어\n \n넌 내게 멋진 여왕이 돼\n어둠에 빠진 나를 구해\n가끔씩은 슬픈 영웅이 돼\n빛이 나 oh\n밤 하늘 아래 별들 같아\n푸른 숲속의 향기 같아\n다 너의 수식어 yeah yeah\n너로 가득 차 있는\n나의 세상 속 oh oh\n난 지금 노래 가사로\n너를 얘기하고 oh oh\n머릿속에 춤추는 멋진 말\nIt's yours It's yours It's yours\n(yeah It's yours 널 위한 것)\n \nSay yeah yeah yeah yeah\nCause you’re my word\n그 어떤 말이 너만큼 많이 빛날까\nIt's yours It's yours It's yours\n모두 다 주고 싶어\nGirl 유치한 생각이 넘쳐나\n주인공은 You \n난 온갖 시를 써 Up all night\noh 눈 감고 누우면\n머릿속에 네가 떠다니고\n내일 무슨 말로\n웃게 할지 고민만 해\n알아 솔직히 It’s crazy oh\n남기고 싶어 모든 곳에\n어떤 말보다 아름답게\n보고 계속 봐도 보고 싶게\n널 말이야 ooh\n뭘로 써봐도 난 부족해\n모든 언어로 남겨둘래\n다 너의 수식어 yeah yeah\n너로 가득 차 있는\n나의 세상 속 oh oh\n난 지금 노래 가사로\n너를 얘기하고 oh oh\n머릿속에 춤추는 멋진 말\nIt's yours It's yours It's yours\n(yeah I

In [3]:
artist_info_file = "artist_infos.jsonl"
with open(artist_info_file, "r", encoding='utf-8') as f:
    lines = f.readlines()
    artist_infos = [json.loads(_) for _ in lines]
    
artist_infos = {_["artist_num"]: _ for _ in artist_infos}
artist_infos["2636017"]

{'artist_num': '2636017',
 '이름': 'PRODUCE X 101',
 '활동년대': '2010 년대',
 '유형': '그룹|남성',
 '장르': '댄스, 발라드',
 '소속사명': 'CJ E&M'}

In [4]:
parsed_chart_path = "parsed_year_chart_html"
jsonl_files = os.listdir(parsed_chart_path)
jsonl_files = [_ for _ in sorted(jsonl_files) if _.endswith("jsonl")]

In [5]:
chart_data = []
for jsonl_file in jsonl_files:
    with open(os.path.join(parsed_chart_path, jsonl_file), "r", encoding='utf-8') as f:
        lines = f.readlines()
        docs = [json.loads(_) for _ in lines]
    chart_data.extend(docs)

In [6]:
len(chart_data)

1898

In [7]:
lasts = []
for line in tqdm(chart_data):
    num_artists = len(line["artists"])
    if num_artists > 1:
        artist_activity_type = "프로젝트"
        artist_id = "|".join([_[0] for _ in line["artists"]])
        artist_name = "|".join([_[1] for _ in line["artists"]])
    elif num_artists == 0:
        artist_activity_type = ""
        artist_id = ""
        artist_name = ""
    else:
        artist_id = line["artists"][0][0]
        artist_name = line["artists"][0][1]
        artist_activity_type = artist_infos.get(artist_id, {}).get("유형", "")

    song_id = line["title_num"]
    song_genre = song_infos.get(song_id, {}).get("장르", "")

    last = {}
    #last["yearweek"] = line["yearweek"]
    last["period"] = line["period"]
    last["ranking"] = line["ranking"]
    last["song_id"] = line["title_num"]
    last["song_name"] = line["title_name"]
    last["song_genre"] = song_genre
    
    last["artist_id"] = artist_id
    last["artist_name"] = artist_name
    last["artist_activity_type"] = artist_activity_type
    
    last["album_id"] = line["album_num"]
    last["album_name"] = line["album_name"]
        
    lasts.append(last)


100%|██████████| 1898/1898 [00:00<00:00, 237168.24it/s]


In [8]:
df = pd.DataFrame(lasts)

In [9]:
df

,period,ranking,song_id,song_name,song_genre,artist_id,artist_name,artist_activity_type,album_id,album_name
0,2005,1,711627,죄와벌,발라드,108363,SG 워너비,그룹|남성,50578,살다가
1,2005,2,732512,겁쟁이,록/메탈,160091,버즈,그룹|남성,300646,Buzz Effect
2,2005,3,837567,사랑했나봐,록/메탈,923,윤도현,솔로|남성,301886,Difference
3,2005,4,711626,살다가,발라드,108363,SG 워너비,그룹|남성,50578,살다가
4,2005,5,711628,광(狂),발라드,108363,SG 워너비,그룹|남성,50578,살다가
...,...,...,...,...,...,...,...,...,...,...
1893,2023,96,31343875,첫사랑,"인디음악, 포크/블루스",2113114,백아,솔로|여성,10210149,첫사랑
1894,2023,97,35668899,딱 10CM만,R&B/Soul,468244|2744750,10CM|BIG Naughty (서동현),프로젝트,11063205,딱 10CM만
1895,2023,98,36595401,"잘 지내자, 우리 (여름날 우리 X 로이킴)",,699632,로이킴,솔로|남성,11280645,"잘 지내자, 우리 (여름날 우리 X 로이킴)"
1896,2023,99,36430774,Allergy,댄스,2137482,(여자)아이들,그룹|여성,11240232,I feel


In [10]:
df.to_excel("melon_year_top100_2005-2023.xlsx", index=False, engine="openpyxl")